# Reference atomic structures

This Notebook gets reference atomic crystal structures from various external sites and compares them to known crystal prototype structures.

**Library imports**

In [1]:
# Standard Python libraries
from __future__ import (print_function, division, absolute_import,
                        unicode_literals)
import os
import glob

# http://www.numpy.org/
import numpy as np

# https://pandas.pydata.org/
import pandas as pd

# https://github.com/usnistgov/atomman
import atomman as am
import atomman.lammps as lmp

# https://github.com/usnistgov/iprPy
import iprPy
print('iprPy version', iprPy.__version__)

iprPy version 0.8.1


## 1. Build unique sets of elements from included potentials

### Load database

In [2]:
database = iprPy.load_database('demo')

### Option #1: Get all elements lists from potentials

In [9]:
elements_set = set()
for potential_record in database.get_records(style='potential_LAMMPS'):
    potential = lmp.Potential(potential_record.content)
    elements = potential.elements()
    elements.sort()
    elements_set.add(' '.join(elements))

### Option #2: Only get elements lists for certain systems

In [3]:
elements_set = set()
elements_set.add('Ag')

## 2. Fetch Materials Project reference structures

### Define get_mp_structures() function

In [4]:
def get_mp_structures(elements, api_key=None, lib_directory=None):
    """
    Accesses the Materials Project and downloads all structures for a list of
    elements as poscar files.
    
    Parameters
    ----------
    elements : list
        A list of element symbols.
    api_key : str, optional
        The user's Materials Project API key. If not given, will use "MAPI_KEY"
        environment variable
    lib_directory : str
        Path to the lib_directory to save the poscar files to.  Default uses
        the iprPy library/dft_structures directory.
    """
    # Function-specific imports
    import pymatgen as pmg
    from pymatgen.ext.matproj import MPRester
    
    # Define subset generator
    def subsets(fullset):
        for i, item in enumerate(fullset):
            yield [item]
            if len(fullset) > 1:
                for subset in subsets(fullset[i+1:]):
                    yield [item] + subset
    
    # Handle lib_directory
    if lib_directory is None:
        lib_directory = os.path.join(os.path.dirname(iprPy.rootdir), 'library', 'ref')
    lib_directory = os.path.abspath(lib_directory)
    
    elements.sort()
    
    # Open connection to Materials Project
    with MPRester(api_key) as m:
        
        # Loop over subsets of elements
        for subelements in subsets(elements):
            
            # Set comp_directory
            elements_string = '-'.join(subelements)
            comp_directory = os.path.join(lib_directory, elements_string)
            if not os.path.isdir(comp_directory):
                os.makedirs(comp_directory)
            
            # Build list of downloaded entries
            have = []
            for fname in glob.iglob(os.path.join(comp_directory, 'mp-*.poscar')):
                have.append(os.path.splitext(os.path.basename(fname))[0])
            #print('Have', len(have), elements_string, 'records')
            
            # Query MP for all entries corresponding to the elements
            entries = m.query({"elements": subelements}, ["material_id"])
            
            # Add entries to the list if not there
            missing = []
            for entry in entries:
                if entry['material_id'] not in have and entry['material_id'] not in missing:
                    missing.append(entry['material_id'])
            #print('Missing', len(missing), elements_string, 'records')
            
            # Download missing entries
            entries = m.query({"material_id": {"$in": missing}}, ['material_id', 'cif'])
            
            # Convert cif to poscar and save
            for entry in entries:
                struct = pmg.Structure.from_str(entry['cif'], fmt='cif')
                struct = pmg.symmetry.analyzer.SpacegroupAnalyzer(struct).get_conventional_standard_structure()
                system = am.load('pymatgen_Structure', struct)
                system = system.normalize()
                structure_file = os.path.join(comp_directory, entry['material_id']+'.poscar')
                system.dump('poscar', f=structure_file)
                print('Added', entry['material_id'])

### Get reference structures

Specify personal Materials Project API key

In [5]:
mp_api_key_location = 'C:\\users\\lmh1\\Documents\\Materials Project\\API key.txt'

with open(mp_api_key_location) as f:
    mp_api_key = f.read()

In [6]:
for elements in elements_set:
    get_mp_structures(elements.split(), api_key=mp_api_key)

C:\Users\lmh1\AppData\Local\Continuum\anaconda3\lib\site-packages\pymatgen\io\cif.py:44: UserWarning: Please install optional dependency pybtex if youwant to extract references from CIF files.
  warnings.warn("Please install optional dependency pybtex if you"


Added mp-10597
Added mp-124
Added mp-8566
Added mp-989737


## 3. Fetch OQMD reference structures

### Define get_oqmd_structures() function

In [7]:
def get_oqmd_structures(elements, lib_directory=None):
    """
    Accesses the Materials Project and downloads all structures for a list of
    elements as poscar files.
    
    Parameters
    ----------
    elements : list
        A list of element symbols.
    lib_directory : str
        Path to the lib_directory to save the poscar files to.  Default uses
        the iprPy library/dft_structures directory.
    """
    # Function-specific imports
    import requests
    
    # Define subset generator
    def subsets(fullset):
        for i, item in enumerate(fullset):
            yield [item]
            if len(fullset) > 1:
                for subset in subsets(fullset[i+1:]):
                    yield [item] + subset
    
    # Get default lib_directory
    if lib_directory is None:
        lib_directory = os.path.join(os.path.dirname(iprPy.rootdir), 'library', 'ref')
    lib_directory = os.path.abspath(lib_directory)
    
    # Set comp_directory
    elements.sort()
    have = []
    for subelements in subsets(elements):
        elements_string = '-'.join(subelements)
        comp_directory = os.path.join(lib_directory, elements_string)
        if not os.path.isdir(comp_directory):
            os.makedirs(comp_directory)
        
        # Build list of downloaded entries
        for fname in glob.iglob(os.path.join(comp_directory, 'oqmd-*.poscar')):
            have.append(os.path.splitext(os.path.basename(fname))[0])
    #print('Have', len(have), 'records')
    
    # Build list of missing OQMD entries
    elements_string = '-'.join(elements)
    
    composition_r = requests.get('http://oqmd.org/materials/composition/' + elements_string)
    composition_html = composition_r.text
    
    missing = []
    count = 0
    while True:
        count += 1
        try:
            start = composition_html.index('href="/materials/entry/') + len('href="/materials/entry/')
        except:
            break
        else:
            end = start + composition_html[start:].index('">')
            entry_number = composition_html[start:end]
            composition_html = composition_html[end+2:]
            entry_id = 'oqmd-'+entry_number
            if entry_id not in have and entry_id not in missing:
                missing.append(entry_id)
        if count > 100:
            raise ValueError('Loop likely infinite')
    #print('Missing', len(missing), 'records')
    
    # Download missing entries
    for entry_id in missing:
        entry_number = entry_id.replace('oqmd-', '')
        entry_r = requests.get('http://oqmd.org/materials/entry/' + entry_number)
        entry_html = entry_r.text
        
        start = entry_html.index('href="/materials/structure/') + len('href="/materials/structure/')
        end = start + entry_html[start:].index('">')
        structure_number = entry_html[start:end]
        
        try:
            structure_url = 'http://oqmd.org/materials/export/conventional/poscar/' + structure_number
            structure_r = requests.get(structure_url)
            structure_r.raise_for_status()
        except:
            try:
                structure_url = 'http://oqmd.org/materials/export/primitive/poscar/' + structure_number
                structure_r = requests.get(structure_url)
                structure_r.raise_for_status()
            except:
                continue
        
        # Save poscar
        poscar = structure_r.text
        system = am.load('poscar', poscar)
        system = system.normalize()
        elements_string = '-'.join(system.symbols)
        structure_file = os.path.join(lib_directory, elements_string, entry_id + '.poscar')
        
        with open(structure_file, 'w') as f:
            f.write(poscar)
        print('Added', entry_id)

### Get reference structures

In [8]:
for elements in elements_set:
    try:
        get_oqmd_structures(elements.split())
    except:
        print('Failed:', elements)

Added oqmd-11876
Added oqmd-676271
Added oqmd-10125
Added oqmd-1215660
Added oqmd-1216016
Added oqmd-1215214
Added oqmd-1215125
Added oqmd-1214591
Added oqmd-1214858
Added oqmd-1214769
Added oqmd-1215838
Added oqmd-1214947
Added oqmd-1215036
Added oqmd-1215571
Added oqmd-1214680
Added oqmd-1215482
Added oqmd-1215927


## 4. Run crystal_space_group calculations on the reference structures

In [9]:
calculation = iprPy.load_calculation('crystal_space_group')
run_directory = iprPy.load_run_directory('demo_1')

In [10]:
input_script = """

# Build load information based on prototype records
buildcombos                 crystalprototype load_file proto

# Build load information based on reference structures
buildcombos                 atomicreference load_file ref

# Specify reference buildcombos limiters (only build for element sets listed)
ref_elements                Ag

# Units that input/output values are in
length_unit                 
pressure_unit               
energy_unit                 
force_unit                  

# Run parameters
symmetryprecision           
primitivecell               
idealcell                   
"""
with open('input_script.in', 'w') as f:
    f.write(input_script)

In [11]:
calculation = iprPy.load_calculation('crystal_space_group')

with open('input_script.in') as f:
    input_dict = iprPy.input.parse(f, singularkeys=calculation.singularkeys)
    
database.prepare(run_directory, calculation, **input_dict)
database.check_records(calculation.record_style)

In database style local at C:\Users\lmh1\Documents\calculations\ipr\demo :
- 40 of style calculation_crystal_space_group
 - 0 are complete
 - 40 still to run
 - 0 issued errors


In [12]:
database.runner(run_directory)
database.check_records(calculation.record_style)

Runner started with pid 8528
No simulations left to run
In database style local at C:\Users\lmh1\Documents\calculations\ipr\demo :
- 40 of style calculation_crystal_space_group
 - 40 are complete
 - 0 still to run
 - 0 issued errors


## 5. Match prototypes to references

### Get results and split into reference and prototype

In [13]:
results_df = database.get_records_df(style=calculation.record_style, full=True, flat=True)

In [14]:
reference_df = results_df[results_df.family+'.poscar'==results_df.load_file].reset_index()
prototype_df = results_df[results_df.family+'.json'==results_df.load_file].reset_index()

### Match based on space group information

In [17]:
try:
    match_df = pd.read_csv('reference_prototype_match.csv')
except:
    match_df = pd.DataFrame()

if len(reference_df) != len(match_df):
    # Match based on Pearson symbol and space group number
    match_df = []
    for reference in reference_df.itertuples():
        match_dict = {}
        match_dict['reference'] = reference.family
        match_dict['site'], match_dict['number'] = reference.family.split('-')
        match_dict['number'] = int(match_dict['number'])
        matches = prototype_df[((reference.pearson_symbol == prototype_df.pearson_symbol)
                               &(reference.spacegroup_number == prototype_df.spacegroup_number))]
        if len(matches) == 1:
            match_dict['prototype'] = matches.iloc[0].family
            match_dict['ref_wykoff'] = reference.wykoff_letters
        elif len(matches) == 0:
            match_dict['prototype'] = np.nan
        else:
            match_dict['prototype'] = 'multiple'
        match_df.append(match_dict)
    match_df = pd.DataFrame(match_df)

    # Check known equivalent Wykoff sites for prototypes
    match_df.loc[(match_df.prototype=='A1--Cu--fcc') & (~match_df.ref_wykoff.isin(['a', 'b'])),
                 'prototype'] = np.nan
    match_df.loc[(match_df.prototype=='A2--W--bcc') & (~match_df.ref_wykoff.isin(['a'])),
                 'prototype'] = np.nan
    match_df.loc[(match_df.prototype=='A3--Mg--hcp') & (~match_df.ref_wykoff.isin(['b', 'c', 'd'])),
                 'prototype'] = np.nan
    match_df.loc[(match_df.prototype=="A3'--alpha-La--double-hcp") & (~match_df.ref_wykoff.isin(['a b', 'a c', 'a d'])),
                 'prototype'] = np.nan
    match_df.loc[(match_df.prototype=='A4--C--dc') & (~match_df.ref_wykoff.isin(['a', 'b'])),
                 'prototype'] = np.nan
    match_df.loc[(match_df.prototype=='A5--beta-Sn') & (~match_df.ref_wykoff.isin(['a', 'b'])),
                 'prototype'] = np.nan
    match_df.loc[(match_df.prototype=='A6--In--bct') & (~match_df.ref_wykoff.isin(['a', 'b'])),
                 'prototype'] = np.nan
    match_df.loc[(match_df.prototype=='A7--alpha-As') & (~match_df.ref_wykoff.isin(['c'])),
                 'prototype'] = np.nan
    match_df.loc[(match_df.prototype=='Ah--alpha-Po--sc') & (~match_df.ref_wykoff.isin(['a', 'b'])),
                 'prototype'] = np.nan
    match_df.loc[(match_df.prototype=='A15--beta-W') & (~match_df.ref_wykoff.isin(['a c', 'a d'])),
                 'prototype'] = np.nan

    # Sort and remove excess information
    match_df = match_df.sort_values(['site', 'number']).reset_index()[['reference', 'prototype']]
    
    # Identify reference compositions
    refdir = os.path.join(iprPy.rootdir, '..', 'library', 'ref')
    match_df['composition'] = [np.nan for i in range(len(match_df))]
    for fname in glob.iglob(os.path.join(refdir, '*', '*.poscar')):
        reference = os.path.splitext(os.path.basename(fname))[0]
        with open(fname) as f:
            lines = f.readlines()
            symbols = lines[5].split()
            counts = list(np.array(lines[6].split(), dtype=int))
            composition = iprPy.analysis.composition(symbols, counts)
            match_df.loc[match_df.reference==reference, 'composition'] = composition
    
    # Save
    match_df.to_csv('reference_prototype_match.csv', index=False)

In [18]:
match_df

,reference,prototype,composition
0,mp-124,A1--Cu--fcc,Ag
1,mp-8566,A3'--alpha-La--double-hcp,Ag
2,mp-10597,A3--Mg--hcp,Ag
3,mp-989737,NaN,Ag
4,oqmd-10125,A3--Mg--hcp,Ag
5,oqmd-11876,A3'--alpha-La--double-hcp,Ag
6,oqmd-676271,A1--Cu--fcc,Ag
7,oqmd-1214591,NaN,Ag
8,oqmd-1214680,NaN,Ag
9,oqmd-1214769,NaN,Ag
